In [1]:
%cd ..

/mnt/SSD_Data/active_projects/transformer_to_lstm


In [2]:
import dataclasses
from pathlib import Path
import warnings

import nlp
import torch
import numpy as np
import torch.nn.functional as F
from transformers import BertForSequenceClassification
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import train_test_split

from pytorch_helper_bot import (
    BaseBot, MovingAverageStatsTrackerCallback,  CheckpointCallback,
    LearningRateSchedulerCallback, MultiStageScheduler, Top1Accuracy,
    LinearLR, Callback
)

from nobita.models import get_sequence_model


try:
    from apex import amp
    APEX_AVAILABLE = True
except ModuleNotFoundError:
    APEX_AVAILABLE = False
# APEX_AVAILABLE = False

In [3]:
CACHE_DIR = Path("cache/")
CACHE_DIR.mkdir(exist_ok=True)

Reference:

    * https://github.com/huggingface/nlp/blob/master/notebooks/Overview.ipynb

In [4]:
class SST2Dataset(torch.utils.data.Dataset):
    def __init__(self, entries_dict):
        super().__init__()
        self.entries_dict = entries_dict
    
    def __len__(self):
        return len(self.entries_dict["label"])
    
    def __getitem__(self, idx):
        return (
            self.entries_dict["input_ids"][idx],
            self.entries_dict["attention_mask"][idx].sum(), #input_lengths
            {
                "label": self.entries_dict["label"][idx], 
                "logits": self.entries_dict["logits"][idx]
            }
        )

In [5]:
train_dict, valid_dict, test_dict = torch.load(str(CACHE_DIR / "distill-dicts.jbl"))

In [6]:
# Instantiate a PyTorch Dataloader around our dataset
train_loader = torch.utils.data.DataLoader(SST2Dataset(train_dict), batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(SST2Dataset(valid_dict), batch_size=64, drop_last=False)
test_loader = torch.utils.data.DataLoader(SST2Dataset(test_dict), batch_size=64, drop_last=False)

In [7]:
ALPHA = 1.
DISTILL_OBJECTIVE = torch.nn.MSELoss()

def distill_loss(logits, targets):
    distill_part = DISTILL_OBJECTIVE(
        logits.reshape(-1), targets["logits"].reshape(-1)
    ) / 2
    classification_part = F.cross_entropy(
        logits, targets["label"]
    )
    return ALPHA * classification_part + (1-ALPHA) * distill_part

In [8]:
bert_model = BertForSequenceClassification.from_pretrained(str(CACHE_DIR / "sst2_bert_uncased")).cpu()

In [9]:
bert_model.bert.embeddings.word_embeddings.weight.shape

torch.Size([30522, 768])

In [10]:
# Note: apex does not support weight dropping
model = get_sequence_model(
    voc_size=bert_model.bert.embeddings.word_embeddings.weight.shape[0],
    emb_size=bert_model.bert.embeddings.word_embeddings.weight.shape[1],
    pad_idx = 0,
    dropoute = 0,
    rnn_hid = 768,
    rnn_layers = 2,
    bidir = True,
    dropouth = 0.25,
    dropouti = 0.25,
    wdrop = 0,
    unit_type = "gru",
    fcn_layers = [512, 2],
    fcn_dropouts = [0.25, 0.25],
    use_attention = True
)

In [11]:
model

SequenceModel(
  (embeddings): BasicEmbeddings(
    (encoder): Embedding(30522, 768, padding_idx=0)
  )
  (encoder): RNNStack(
    (rnns): ModuleList(
      (0): GRU(768, 384, bidirectional=True)
      (1): GRU(768, 384, bidirectional=True)
    )
    (dropouti): LockedDropout()
    (dropouths): ModuleList(
      (0): LockedDropout()
      (1): LockedDropout()
    )
  )
  (fcn): AttentionFCN(
    (attention): Attention(768, return attention=False)
    (layers): ModuleList(
      (0): LinearBlock(
        (lin): Linear(in_features=768, out_features=512, bias=True)
        (drop): Dropout(p=0.25, inplace=False)
        (bn): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): LinearBlock(
        (lin): Linear(in_features=512, out_features=2, bias=True)
        (drop): Dropout(p=0.25, inplace=False)
        (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
)

In [12]:
# Copy the embedding weights to the LSTM model
try:
    model.embeddings.encoder.emb.weight.data = bert_model.bert.embeddings.word_embeddings.weight.data
except:
    model.embeddings.encoder.weight.data = bert_model.bert.embeddings.word_embeddings.weight.data

In [13]:
# Freeze the embedding layer
for param in model.embeddings.encoder.parameters():
    param.requires_grad = False

In [14]:
model = model.cuda()

In [15]:
# Use only leaf tensors
parameters = [x for x in model.parameters() if x.is_leaf and x.requires_grad]

In [16]:
del bert_model

In [17]:
optimizer = torch.optim.Adam(parameters, lr=1e-3, betas=(0.8, 0.99))
# optimizer = torch.optim.RMSprop(parameters, lr=0.01)

In [18]:
if APEX_AVAILABLE:
    model, optimizer = amp.initialize(
        model, optimizer, opt_level="O1"
    )

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [19]:
class TransposeCallback(Callback):
    def on_batch_inputs(self, bot, input_tensors, targets):
        input_tensors = [input_tensors[0].transpose(1, 0), input_tensors[1]]
        return input_tensors, targets

In [20]:
class DistillTop1Accuracy(Top1Accuracy):
    def __call__(self, truth, pred):
        truth = truth["label"]
        return super().__call__(truth, pred)

In [21]:
total_steps = len(train_loader) * 10

checkpoints = CheckpointCallback(
    keep_n_checkpoints=1,
    checkpoint_dir=CACHE_DIR / "distill_model_cache/",
    monitor_metric="loss"
)
lr_durations = [
    int(total_steps*0.2),
    int(np.ceil(total_steps*0.8))
]
break_points = [0] + list(np.cumsum(lr_durations))[:-1]
callbacks = [
    MovingAverageStatsTrackerCallback(
        avg_window=len(train_loader) // 8,
        log_interval=len(train_loader) // 10
    ),
    LearningRateSchedulerCallback(
        MultiStageScheduler(
            [
                LinearLR(optimizer, 0.01, lr_durations[0]),
                CosineAnnealingLR(optimizer, lr_durations[1])
            ],
            start_at_epochs=break_points
        )
    ),
    checkpoints,
    TransposeCallback()
]
    
bot = BaseBot(
    log_dir = CACHE_DIR / "distill_logs",
    model=model, 
    train_loader=train_loader,
    valid_loader=valid_loader, 
    clip_grad=10.,
    optimizer=optimizer, echo=True,
    criterion=distill_loss,
    callbacks=callbacks,
    pbar=False, use_tensorboard=False,
    use_amp=APEX_AVAILABLE,
    metrics=(DistillTop1Accuracy(),)
)

[INFO][06/15/2020 21:01:38] SEED: 9293
[INFO][06/15/2020 21:01:38] # of parameters: 29,156,610
[INFO][06/15/2020 21:01:38] # of trainable parameters: 5,715,714


In [22]:
print(total_steps)

bot.train(
    total_steps=total_steps,
    checkpoint_interval=len(train_loader) // 2
)
bot.load_model(checkpoints.best_performers[0][1])
checkpoints.remove_checkpoints(keep=0)

[INFO][06/15/2020 21:01:38] Optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.8, 0.99)
    eps: 1e-08
    initial_lr: 0.001
    lr: 0.001
    weight_decay: 0
)
[INFO][06/15/2020 21:01:38] Batches per epoch: 1053
[INFO][06/15/2020 21:01:38] ====================Epoch 1====================


10530


[INFO][06/15/2020 21:01:40] Step   105 | loss 1.03384137 | lr: 5.94e-05 | 0.019s per step
[INFO][06/15/2020 21:01:42] Step   210 | loss 0.85595699 | lr: 1.09e-04 | 0.018s per step
[INFO][06/15/2020 21:01:44] Step   315 | loss 0.73923854 | lr: 1.58e-04 | 0.018s per step
[INFO][06/15/2020 21:01:46] Step   420 | loss 0.67060309 | lr: 2.08e-04 | 0.019s per step
[INFO][06/15/2020 21:01:47] Step   525 | loss 0.59762839 | lr: 2.57e-04 | 0.018s per step
[INFO][06/15/2020 21:01:48] Metrics at step 526:
[INFO][06/15/2020 21:01:48] loss: 0.44859088
[INFO][06/15/2020 21:01:48] accuracy: 80.50%
[INFO][06/15/2020 21:01:50] Step   630 | loss 0.57235740 | lr: 3.06e-04 | 0.021s per step
[INFO][06/15/2020 21:01:52] Step   735 | loss 0.54173837 | lr: 3.56e-04 | 0.019s per step
[INFO][06/15/2020 21:01:54] Step   840 | loss 0.52237535 | lr: 4.05e-04 | 0.019s per step
[INFO][06/15/2020 21:01:56] Step   945 | loss 0.48057210 | lr: 4.54e-04 | 0.019s per step
[INFO][06/15/2020 21:01:58] Step  1050 | loss 0.453

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 262144.0


[INFO][06/15/2020 21:03:34] Step  6090 | loss 0.20136125 | lr: 5.43e-04 | 0.019s per step


...

[INFO][06/15/2020 21:03:36] Step  6195 | loss 0.20547779 | lr: 5.24e-04 | 0.019s per step
[INFO][06/15/2020 21:03:38] Step  6300 | loss 0.19286398 | lr: 5.04e-04 | 0.018s per step
[INFO][06/15/2020 21:03:39] Metrics at step 6312:
[INFO][06/15/2020 21:03:39] loss: 0.36581496
[INFO][06/15/2020 21:03:39] accuracy: 85.32%
[INFO][06/15/2020 21:03:39] ====================Epoch 7====================
[INFO][06/15/2020 21:03:40] Step  6405 | loss 0.18320435 | lr: 4.84e-04 | 0.019s per step
[INFO][06/15/2020 21:03:42] Step  6510 | loss 0.18189077 | lr: 4.65e-04 | 0.019s per step
[INFO][06/15/2020 21:03:44] Step  6615 | loss 0.17367351 | lr: 4.45e-04 | 0.018s per step
[INFO][06/15/2020 21:03:46] Step  6720 | loss 0.17666738 | lr: 4.26e-04 | 0.018s per step
[INFO][06/15/2020 21:03:48] Step  6825 | loss 0.17735529 | lr: 4.07e-04 | 0.018s per step
[INFO][06/15/2020 21:03:48] Metrics at step 6838:
[INFO][06/15/2020 21:03:48] loss: 0.34449610
[INFO][06/15/2020 21:03:48] accuracy: 87.84%
[INFO][06/15/2

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0


[INFO][06/15/2020 21:04:14] Step  8190 | loss 0.15647669 | lr: 1.79e-04 | 0.020s per step
[INFO][06/15/2020 21:04:16] Step  8295 | loss 0.15544379 | lr: 1.64e-04 | 0.020s per step
[INFO][06/15/2020 21:04:18] Step  8400 | loss 0.15439361 | lr: 1.50e-04 | 0.019s per step
[INFO][06/15/2020 21:04:19] Metrics at step 8416:
[INFO][06/15/2020 21:04:19] loss: 0.32783247
[INFO][06/15/2020 21:04:19] accuracy: 87.16%
[INFO][06/15/2020 21:04:19] ====================Epoch 9====================


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0


[INFO][06/15/2020 21:04:20] Step  8505 | loss 0.15097086 | lr: 1.36e-04 | 0.020s per step
[INFO][06/15/2020 21:04:22] Step  8610 | loss 0.14910909 | lr: 1.23e-04 | 0.019s per step
[INFO][06/15/2020 21:04:24] Step  8715 | loss 0.14713163 | lr: 1.10e-04 | 0.018s per step
[INFO][06/15/2020 21:04:26] Step  8820 | loss 0.14970109 | lr: 9.85e-05 | 0.018s per step
[INFO][06/15/2020 21:04:28] Step  8925 | loss 0.14161665 | lr: 8.71e-05 | 0.019s per step
[INFO][06/15/2020 21:04:28] Metrics at step 8942:
[INFO][06/15/2020 21:04:28] loss: 0.32466900
[INFO][06/15/2020 21:04:28] accuracy: 88.07%
[INFO][06/15/2020 21:04:30] Step  9030 | loss 0.15042173 | lr: 7.64e-05 | 0.019s per step
[INFO][06/15/2020 21:04:32] Step  9135 | loss 0.14744298 | lr: 6.63e-05 | 0.019s per step
[INFO][06/15/2020 21:04:34] Step  9240 | loss 0.14847580 | lr: 5.69e-05 | 0.018s per step
[INFO][06/15/2020 21:04:36] Step  9345 | loss 0.14032037 | lr: 4.82e-05 | 0.018s per step
[INFO][06/15/2020 21:04:38] Step  9450 | loss 0.14

In [23]:
# TARGET_DIR = CACHE_DIR / "sst2_bert_uncased"
# TARGET_DIR.mkdir(exist_ok=True)
# bot.model.save_pretrained(TARGET_DIR)

In [24]:
bot.eval(valid_loader)

{'loss': (0.29762899930324027, '0.29762900'),
 'accuracy': (-0.8830275229357798, '88.30%')}

In [25]:
bot.eval(test_loader)

{'loss': (0.32853356796667116, '0.32853357'),
 'accuracy': (-0.8646788990825688, '86.47%')}

In [26]:
# tokenizer.pad_token_id